# $target $night

In [ ]:
%pylab inline

In [ ]:
from muscat2ta.transitanalysis import TransitAnalysis as TA

## Plot the field

![]($field)

## Transit analysis initialisation

The transit analysis is done with a `TransitAnalysis` class that takes the root data directory (`droot`), target name (`target`), observing night (`night`), target ID (`tid`), and a list of comparison star IDs (`cids`). 

The data should be organized so that the photometry files are in `droot/target/night`.

In [ ]:
ta = TA('$target', '$night', tid=TID, cids=CIDS, use_opencl=False, radius_ratio='achromatic')

## Light curve cleanup
### Plot the target and the n brightest stars

These plots show the raw absolute light curves for the target and the brightest stars in the field. The plots will not be saved, but help when choosing the reference stars and deciding if the light curve needs to be trimmed.

In [ ]:
ta.plot_raw_light_curves(ylim=(0.95, 1.05));

**Light curve trimming:**

If the light curve contains spans of bad data, they can be trimmed away usin `ta.cut()`. The method will remove the data **between** `tstart` and `tend`. If you want to cut all the data from beginning till `START_TIME`, you can call the method as

    ta.lpf.cut(tend=START_TIME, apply=False)

if you want to cut all the data after `END_TIME`, you can call the method as

    ta.lpf.cut(tstart=END_TIME, apply=False)
    
and if you want to remove the data between times `T1` and `T2`, you can call the method as 
   
    ta.lpf.cut(tstart=T1, tend=t2, apply=False)

*Note:* You need to set `apply=True` to actually apply the outlier removal. By default the command only plots what would be removed if `apply` would be `True`.

In [ ]:
ta.lpf.cut(tstart=-inf, tend=inf, apply=False)

**Outlier removal:** The pipeline comes with several ways to remove outliers, of which `ta.lpf.apply_normalized_limits(aid, lower, upper, apply, plot, npoly, iterations)` is the most user friendly. The method removes points with relative normalised flux smaller than `lower` and larger than `upper`.

*Note:* You need to set `apply=True` to actually apply the outlier removal. By default the command only plots what would be removed if `apply` would be `True`.

In [ ]:
ta.apply_normalized_limits(4, lower=0.9, upper=1.1, apply=False, plot=True, npoly=2, iterations=5)

**Downsampling:** the light curves can be downsampled (binned) to 30-60 second exposures in most cases without compromising the quality of the analysis. This can speed up the analysis significantly, and is especially recommended with bright TESS targets.

In [ ]:
ta.downsample(60)

## Set the parameter priors

The basic `TransitAnalysis` class doesn't know how the automatically set the priors for the period, transit centre, radius ratio, etc., but you need to set these manually. The priors can be set using the `ta.set_prior` method as

    ta.set_prior(PARAM, PRIOR, A, B)
    
where `PARAM` is the parameter name, `PRIOR` is either `"NP"` for a normal prior or `"UP"` for a uniform prior, and `A` and `B` are the prior parameters (`PRIOR` can also be a `Prior` object, but this is for more advanced use). For example, setting a normal prior on the orbital period with a mean of 1.5 and an std of 0.1 is done as

    ta.set_prior('p', 'NP', 1.5, 0.1)
    
and setting a uniform prior from 0.5 to 10 g cm$^{-3}$ for the stellar density is done as

    ta.set_prior('rho', 'UP', 0.5, 10.0)
    
First, let's print the parameters and the priors

In [ ]:
ta.lpf.print_parameters(1)

Next, we change all the priors that need to be changed. Probably at least the transit centre and orbital period.

In [ ]:
ta.set_prior('tc', 'NP', 0.0, 0.001) # Change me! Transit centre prior
ta.set_prior('p', 'NP',  1.0, 0.001) # Change me! orbital period prior

Finally, let's print the parameters again to make sure our changes are correct.

In [ ]:
ta.lpf.print_parameters(1)

## Global optimisation

The global optimisation step aims to find the best combination of reference stars (from the set of reference star IDs given in the initialisation), target and reference apertures, global posterior maximum for the transit model, and the trends modelled as a linear combination of covariates.

In [ ]:
ta.optimize(niter=1000)

In [ ]:
ta.plot_light_curve();

The photometry should be frozen with `freeze_photometry()` after a good solution has been found. This fixes the reference stars and the target and reference apertures to the best-fit solution, and makes the MCMC sampling significantly more efficient.

In [ ]:
ta.freeze_photometry()

In [ ]:
ta.optimize(niter=1000)

In [ ]:
ta.plot_light_curve();

## MCMC Sampling

In [ ]:
ta.sample(niter=1000, thin=10, repeats=3)

In [ ]:
ta.plot_light_curve(method='mc', figsize=(13, 8));

In [ ]:
ta.plot_basic_posteriors();

## Saving

In [ ]:
ta.save()

In [ ]:
ta.save_fits()

---

<center> &copy; 2019 Hannu Parviainen </center>